# Cleaning and structuring operational data from municipal inspections in Miami-Dade: from raw to usable
_Author: Frank R Gamboa | Scio Labs 2025 | August 2025_

### **Importing the libraries needed for the analysis**
We will use `pandas` for data manipulation and `numpy` for numerical operations.

In [240]:
# Import standard libraries for data handling

import pandas as pd
import numpy as np
import re

### **Loading the original dataset from our GitHub repository**


In [241]:
# Loading the dataset directly from the raw github url

url = "https://raw.githubusercontent.com/frankrgamboa/miami-inspections-cleaning/refs/heads/main/Inspections.csv"
df = pd.read_csv(url)

# Quick look at the first rows

df.head(8)

,date,permit_number,sequence_number,input_order,inspector_id,first_name,last_name,photo,super_name,super_email,super_phone,dir_name,dir_email,dir_phone,inspection_description,job_site_address,permit_type,request_date,disp_description,ObjectId
0,2025/06/05 04:00:00+00,2025006752,1,30,E312568,Eusebio,Aguilar-Godoy,images\jpgn\Eusebio Godoy.jpg,Mario Montero,Mario.Montero@miamidade.gov,(786) 315-2224,Jose Lara Gomez,Jose.LaraGomez@miamidade.gov,(786) 315-2084,ROUGH,1611 NW 12 AVE,MECH,2025/06/05 04:00:00+00,NOT INSPECTED,1
1,2025/06/05 04:00:00+00,2023056382,27,194,E406957,Alexander,Bedoya,E406957.jpg,Robert Fatora,Robert.Fatora@miamidade.gov,(786) 315-2218,Gilberto Martin,Gilberto.Martin@miamidade.gov,(786) 315-2082,INSULATION INSPECTIONS,1700 NE 145 ST,BLDG,2025/06/05 04:00:00+00,NOT INSPECTED,2
2,2025/06/05 04:00:00+00,2024058290,3,27,E314249,Luigi,Chinea,images\jpgn\Luigi Chinea.jpg,Mario Montero,Mario.Montero@miamidade.gov,(786) 315-2224,Jose Lara Gomez,Jose.LaraGomez@miamidade.gov,(786) 315-2084,ROUGH,3100 SW 62 AVE,MECH,2025/06/05 04:00:00+00,NOT INSPECTED,3
3,2025/06/05 04:00:00+00,2024056228,4,99,LMOLINA,Lawrence,Molina,LMOLINA.jpg,Alberto Blaya,Alberto.Blaya@miamidade.gov,(786) 315-2421,Matthew Agostini,Matthew.Agostini@miamidade.gov,(786) 315-2083,ROUGH,1611 NW 12 AVE,ELEC,2025/06/05 04:00:00+00,NOT INSPECTED,4
4,2025/06/05 04:00:00+00,2025028446,7,89,E321127,Angel,Castro,E321127.jpg,Robert Fatora,Robert.Fatora@miamidade.gov,(786) 315-2218,Gilberto Martin,Gilberto.Martin@miamidade.gov,(786) 315-2082,BUCK AND FASTENER,9907 SW 164 PL,BLDG,2025/06/05 04:00:00+00,NOT INSPECTED,5
5,2025/06/05 04:00:00+00,2024073267,2,4,E316554,Barnette K,Gunn,images\jpgn\Barnett Gunn.jpg,Bernard Chitolie,Bernard.Chitolie@miamidade.gov,(786) 315-2419,Jose Lara Gomez,Jose.LaraGomez@miamidade.gov,(786) 315-2084,ROUGH,7389 SW 45 ST,PLUM,2025/06/05 04:00:00+00,NOT INSPECTED,6
6,2025/06/05 04:00:00+00,2024022578,1,46,E313999,Kevin,OConnor,images\jpgn\Kevin OConnor.jpg,Bernard Chitolie,Bernard.Chitolie@miamidade.gov,(786) 315-2419,Jose Lara Gomez,Jose.LaraGomez@miamidade.gov,(786) 315-2084,ROUGH,1611 NW 12 AVE,PLUM,2025/06/05 04:00:00+00,NOT INSPECTED,7
7,2025/06/05 04:00:00+00,2025046727,8,22,E314249,Luigi,Chinea,images\jpgn\Luigi Chinea.jpg,Mario Montero,Mario.Montero@miamidade.gov,(786) 315-2224,Jose Lara Gomez,Jose.LaraGomez@miamidade.gov,(786) 315-2084,FINAL,3400 SW 100 AVE,MECH,2025/06/05 04:00:00+00,NOT INSPECTED,8


### **First looks at the data frame and its columns**
The Pandas .info() method allows you to have an overview of the data frame, with a summary of the dimensions, values per column, number of nulls, etc.

In [242]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7844 entries, 0 to 7843
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   date                    7844 non-null   object
 1   permit_number           7844 non-null   int64 
 2   sequence_number         7844 non-null   int64 
 3   input_order             7844 non-null   int64 
 4   inspector_id            7844 non-null   object
 5   first_name              7844 non-null   object
 6   last_name               7844 non-null   object
 7   photo                   7841 non-null   object
 8   super_name              7844 non-null   object
 9   super_email             7844 non-null   object
 10  super_phone             7844 non-null   object
 11  dir_name                7844 non-null   object
 12  dir_email               7844 non-null   object
 13  dir_phone               7844 non-null   object
 14  inspection_description  7844 non-null   object
 15  job_

### **Observations from `.info()` and `.head()`**


---


#### After running `df.info()` and inspecting the first rows, I noticed the following about columns:



- Only two columns have null values: `job_site_address` and `photo`, with 2 and 3 respectively.
-The `date` column is not in `datetime` format. The time associated with each inspection appears to be selected by default or automatically.
-During the review of `inspector_id` column, we found that some IDs start with the letter `E` followed by numbers; others appear to be full letter-based usernames (e.g., `LMOLINA`). This inconsistency suggests a lack of standardization in how IDs were recorded.
-The columns `first_name` and `last_name`, which refer to the names of the inspectors, can be joined into a single column `first_name + last_name = full_name` without losing information and reducing dimensionality.
- The `photo` column contains the names of the photos corresponding to each inspection, saved in `.jpg` format, which cannot even be accessed. This column should be deleted.
-The `super_name` and `dir_name` columns appear to contain the names of the county department supervisors/directors responsible for these inspections; they may contain values as proxy variables later. Do not delete these columns for the time being; no unusual data is apparent in these columns at first glance.
-The columns `super_email`, `super_phone`, `dir_phone`, and `dir_email` contain the email and phone numbers of supervisors/directors. These columns will be removed because they contain private data and have no real analytical value (the names are sufficient).
-The inspection_description column appears to contain information about the type of inspection performed. It seems very useful.
-The column `job_site_address` displays only county addresses with street and number, with no ZIP codes or cities within the county.
-The `permit_type` column contains the different types of permit for which each inspection was performed, highly useful.
-The columns `request_date` and `ObjectId` will be eliminated directly: the former suffers from the same problem as the column `date`; in fact, it seems to reproduce the same values, which makes it useless for analysis; the latter seems to contain unique values for each inspection, like an `index`, which increases dimensionality and creates unnecessary bias.
-The column `disp_description` appears to contain the results of the inspections (e.g., `APPROVED`, `REJECTED`, etc.). The name seems unintuitive; it will be changed to `result`. This appears to be the most important variable around which the quantitative analyses will revolve.


### **Deleting the first columns of the data frame**
We'll delete several columns at once: `photo`, `request_date`, `ObjectId`, `super_email`, `dir_email`, `super_phone`, `dir_phone`, and `inspector_id`. We will delete the latter for several reasons:
* It is not standardized, as we mentioned before.
* The information in this column overlaps with the inspectors name, as the column essentially represents an ID assigned to each inspector. Even if multiple IDs were linked to the same inspector name, there are only two likely scenarios: the inspections differ in terms of `permit_type` and/or `inspection_description`; or, the inspections occurred at different `job_site_addresses` — in which case, the discrepancy likely stems from a data entry or system error that we cannot resolve and is not worth addressing for this analysis.

Additionally, we will create a new column to reduce the dimensionality coming from `first_name + last_name`


In [245]:
# Create a copy of the dataframe df

df_copy = df.copy()

# Create a new column joining first_name with last_name into a single string

df_copy['full_name'] = df_copy['first_name'] + ' ' + df_copy['last_name']

# The .drop() method will remove the problematic columns from the copy, with parameter inplace = True,
# which means it changes the dataframe itself (this is why we made a copy)

df_copy.drop(columns=[
    'photo', 'request_date', 'ObjectId', 'super_email', 'dir_email',
    'super_phone', 'dir_phone', 'inspector_id',
    'first_name', 'last_name',
    ], inplace=True)


In [246]:
# Preview the cleaned structure

df_copy.head(8)

,date,permit_number,sequence_number,input_order,super_name,dir_name,inspection_description,job_site_address,permit_type,disp_description,full_name
0,2025/06/05 04:00:00+00,2025006752,1,30,Mario Montero,Jose Lara Gomez,ROUGH,1611 NW 12 AVE,MECH,NOT INSPECTED,Eusebio Aguilar-Godoy
1,2025/06/05 04:00:00+00,2023056382,27,194,Robert Fatora,Gilberto Martin,INSULATION INSPECTIONS,1700 NE 145 ST,BLDG,NOT INSPECTED,Alexander Bedoya
2,2025/06/05 04:00:00+00,2024058290,3,27,Mario Montero,Jose Lara Gomez,ROUGH,3100 SW 62 AVE,MECH,NOT INSPECTED,Luigi Chinea
3,2025/06/05 04:00:00+00,2024056228,4,99,Alberto Blaya,Matthew Agostini,ROUGH,1611 NW 12 AVE,ELEC,NOT INSPECTED,Lawrence Molina
4,2025/06/05 04:00:00+00,2025028446,7,89,Robert Fatora,Gilberto Martin,BUCK AND FASTENER,9907 SW 164 PL,BLDG,NOT INSPECTED,Angel Castro
5,2025/06/05 04:00:00+00,2024073267,2,4,Bernard Chitolie,Jose Lara Gomez,ROUGH,7389 SW 45 ST,PLUM,NOT INSPECTED,Barnette K Gunn
6,2025/06/05 04:00:00+00,2024022578,1,46,Bernard Chitolie,Jose Lara Gomez,ROUGH,1611 NW 12 AVE,PLUM,NOT INSPECTED,Kevin OConnor
7,2025/06/05 04:00:00+00,2025046727,8,22,Mario Montero,Jose Lara Gomez,FINAL,3400 SW 100 AVE,MECH,NOT INSPECTED,Luigi Chinea


### **Handling null/missing values in the `address` column**
By deleting the `photo` column, we eliminate three missing values in the process. We still have two `NaN` values in the data frame, specifically in the `address` column. Since this is a specific address, we can't fill these entries directly; eventually, if the `address` column proves useful, we can perform cross-variable analysis to see if variables like `permit_number`, `date`, `permit`, and `inspector` can determine the address in these two cases. Therefore, we will control the flow of the analysis without deleting these two rows for now.

### **Structural Insight: `permit_type` determines the Organizational Hierarchy**

Through group analysis, we uncovered a strong hierarchical relationship between the following columns:

- Each `permit_type` is associated with a single `dir_name` (division director)
- Each `dir_name` oversees one or more `super_name`s (supervisors)
- Almost all `super_name` are unique to their `dir_name`, with only one exception.

This suggests a near-deterministic relationship:
> Knowing the `permit_type` allows us to infer the responsible division (`dir_name`) and the supervisor (`super_name`).

#### Implications:
- These columns exhibit *functional redundancy*. One can be removed without losing information if dimensionality reduction is needed.
- Data validation rules can be established: if a `permit_type` appears under multiple directors or supervisors, it signals a data inconsistency.
- Hierarchical filtering or grouping by organizational role becomes easier and logically sound.

In [247]:
# Use the .groupby() method to obtain a grouping of the directors (higher in the hierarchy)
# with respect to each grouping of the type (super_name, permit_type)

df_copy.groupby('dir_name')[['super_name','permit_type']].value_counts()

dir_name           super_name           permit_type
Fredy  Tain        Robert  Fatora       STRT             34
Gilberto  Martin   Robert  Fatora       BLDG           2512
                   Freddy  Semino       ROOF           1409
Jose  Lara  Gomez  Bernard  Chitolie    PLUM           1752
                   Mario  Montero       MECH            437
                                        boil             74
Matthew  Agostini  Alberto  Blaya       ELEC           1469
                                        ELET             30
Rick  Roig         Beverly  Washington  ZONE            127
Name: count, dtype: int64

In [248]:
# Create a dictionary, which serves to save this functional relationship between
# permit_type and (super_name, dir_name)

hierarchy = {
    'ROOF':    ('Freddy Semino',     'Gilberto Martin'),
    'BLDG':    ('Robert Fatora',     'Gilberto Martin'),
    'PLUM':    ('Bernard Chitolie',  'Jose Lara Gomez'),
    'MECH':    ('Mario Montero',     'Jose Lara Gomez'),
    'boil':    ('Mario Montero',     'Jose Lara Gomez'),
    'ELEC':    ('Alberto Blaya',     'Matthew Agostini'),
    'ELET':    ('Alberto Blaya',     'Matthew Agostini'),
    'ZONE':    ('Beverly Washington','Rick Roig'),
    'STRT':    ('Robert Fatora',     'Fredy Tain'),
}

This hierarchy variable stored in a dictionary allows this relationship to be reconstructed if necessary at a later time.

In [249]:
# Deleting the columns super_name and dir_name after those analysis

df_copy1 = df_copy.drop(columns=['super_name', 'dir_name'])

# Preview the most recent data frame

df_copy1.head(8)

,date,permit_number,sequence_number,input_order,inspection_description,job_site_address,permit_type,disp_description,full_name
0,2025/06/05 04:00:00+00,2025006752,1,30,ROUGH,1611 NW 12 AVE,MECH,NOT INSPECTED,Eusebio Aguilar-Godoy
1,2025/06/05 04:00:00+00,2023056382,27,194,INSULATION INSPECTIONS,1700 NE 145 ST,BLDG,NOT INSPECTED,Alexander Bedoya
2,2025/06/05 04:00:00+00,2024058290,3,27,ROUGH,3100 SW 62 AVE,MECH,NOT INSPECTED,Luigi Chinea
3,2025/06/05 04:00:00+00,2024056228,4,99,ROUGH,1611 NW 12 AVE,ELEC,NOT INSPECTED,Lawrence Molina
4,2025/06/05 04:00:00+00,2025028446,7,89,BUCK AND FASTENER,9907 SW 164 PL,BLDG,NOT INSPECTED,Angel Castro
5,2025/06/05 04:00:00+00,2024073267,2,4,ROUGH,7389 SW 45 ST,PLUM,NOT INSPECTED,Barnette K Gunn
6,2025/06/05 04:00:00+00,2024022578,1,46,ROUGH,1611 NW 12 AVE,PLUM,NOT INSPECTED,Kevin OConnor
7,2025/06/05 04:00:00+00,2025046727,8,22,FINAL,3400 SW 100 AVE,MECH,NOT INSPECTED,Luigi Chinea


### **Renaming columns for better analysis**
We will simply rename the columns we find less intuitive, so it is more visually clear. Remember that we will eventually use the cleaned data for machine learning and visualizations.

In [250]:
# Create a dictionary that stores the new column names that we will later use to iterate through

rename_dict = {
    'disp_description':'result',
    'job_site_address':'address',
    'permit_type':'permit',
    'full_name':'inspector',
    'inspection_description':'inspection_type'
}

# Only rename if the original column exists. This avoids errors with idempotent methods like .rename()

df_copy1.rename(columns = {k: v for k, v in rename_dict.items() if k in df_copy1.columns}, inplace = True)

# Preview the new data frame

df_copy1.head(8)

,date,permit_number,sequence_number,input_order,inspection_type,address,permit,result,inspector
0,2025/06/05 04:00:00+00,2025006752,1,30,ROUGH,1611 NW 12 AVE,MECH,NOT INSPECTED,Eusebio Aguilar-Godoy
1,2025/06/05 04:00:00+00,2023056382,27,194,INSULATION INSPECTIONS,1700 NE 145 ST,BLDG,NOT INSPECTED,Alexander Bedoya
2,2025/06/05 04:00:00+00,2024058290,3,27,ROUGH,3100 SW 62 AVE,MECH,NOT INSPECTED,Luigi Chinea
3,2025/06/05 04:00:00+00,2024056228,4,99,ROUGH,1611 NW 12 AVE,ELEC,NOT INSPECTED,Lawrence Molina
4,2025/06/05 04:00:00+00,2025028446,7,89,BUCK AND FASTENER,9907 SW 164 PL,BLDG,NOT INSPECTED,Angel Castro
5,2025/06/05 04:00:00+00,2024073267,2,4,ROUGH,7389 SW 45 ST,PLUM,NOT INSPECTED,Barnette K Gunn
6,2025/06/05 04:00:00+00,2024022578,1,46,ROUGH,1611 NW 12 AVE,PLUM,NOT INSPECTED,Kevin OConnor
7,2025/06/05 04:00:00+00,2025046727,8,22,FINAL,3400 SW 100 AVE,MECH,NOT INSPECTED,Luigi Chinea


### **Analyzing the columns `inspection_type` and `result`**
The `inspection_type` and `result` columns suffer from a very common problem within categorical variables: **high categorical cardinality with inconsistent semantics** or **semi-structured free text**. This occurs when there are many distinct categories (**high cardinality**), some of which are only slightly different versions of each other (e.g., differences in capitalization, spaces, abbreviations), and the underlying meaning is the same thing written in different and confusing ways. This is a case of **semantic noise**; we will model this problem to achieve text standardization.

#### Grouping `result` into Standard Categories

The raw `result` column contains dozens of unique inspection outcomes, often inconsistent or too granular for analysis. To simplify the results while preserving meaning, I decided to group them into four general categories:

- **APPROVED**: Fully or partially approved inspections.
- **REJECTED**: All forms of disapproval, including corrections required or plan issues.
- **CANCELLED**: Explicit cancellations (manual, internet, or weather-related).
- **OTHER**: Cases that cannot be clearly assigned to any of the above (e.g., “Demolished”, “Carry Over”, “No Progress”).

This mapping was done manually after reviewing the most frequent values and their meaning.


In [251]:
# Normalize a string through this function 'normalizing'

def normalizing(S):
    if pd.isna(S):
        return ''
    S = str(S).upper().strip()  # uppercase + delete spaces
    S = re.sub(r'\s+', ' ', S)
    return S

# Passing 'normalizing' through 'result' with .apply()

df_copy1['result'] = df_copy1['result'].apply(normalizing)

# Manual mapping dictionary

result_map = {
    # --------------------------APPROVED ------------------------------
    'APPROVED': 'APPROVED',
    'APPROVED PARTIAL': 'APPROVED',
    'PARTIAL APPROVE COMPLX STRUCT': 'APPROVED',
    'APPRVD PARTIAL/EXTRA INSP REQ': 'APPROVED',
    'INSP. & APPROVED BY ARCHITECT': 'APPROVED',
    'INSP. & APPROVED BY ENGINEER': 'APPROVED',
    'PERM BY AFFDVIT/CORRECT REQ': 'APPROVED',
    'APPROVED PLANS REQ.': 'APPROVED',
    'PARTIAL\\ APPROVE TRAD PENDING': 'APPROVED',
    'COMPLETED': 'APPROVED',

    # -------------------------- REJECTED ------------------------------
    'REJECTED NOT ACCESSIBLE': 'REJECTED',
    'REJECTED PLAN & PERMIT MISSING': 'REJECTED',
    'REJECTED WORK CONCEALED': 'REJECTED',
    'REJECTED NOT TO CODE': 'REJECTED',
    'REJECTED REVISE PLANS REQ': 'REJECTED',
    'REJECTED , NOTICE COMMENCEMENT': 'REJECTED',
    'REJECTED APPROVED PLANS REQ.': 'REJECTED',
    'REJECTED MULTI INSPECTED BLDG': 'REJECTED',
    'REJECTED FOR POOL BARRIER REQ': 'REJECTED',
    'REJECTED SUPPLEMENT PERMIT REQ': 'REJECTED',
    'CORRECTIONS REQUIRED': 'REJECTED',

    # -------------------------- CANCELLED -------------------------------
    'INSPECTION CANCELLATION': 'CANCELLED',
    'CANCELLATION BY INTERNET': 'CANCELLED',
    'NOT PERFORMED. BAD WEATHER': 'CANCELLED',

    # ---------------------------- OTHER ----------------------------------
    'NOT READY FOR INSPECTION': 'OTHER',
    'NOT INSPECTED': 'OTHER',
    'CARRY OVER': 'OTHER',
    'FIELD CHECK ONLY': 'OTHER',
    'INSPECTION FOR QTY ASSURANCE': 'OTHER',
    'INSPECT QTY ASSURANCE PARTIAL': 'OTHER',
    'DEMOILISHED': 'OTHER',
    'NO PROGRESS': 'OTHER',
    'VIRTUAL/INSP SEE COMMENT&CALL': 'OTHER',
    'INACTIVE': 'OTHER',
    'UNABLE TO LOCATE/ADD/APT/SPACE': 'OTHER',
    'REFERRED TO REGULATION': 'OTHER'
}


# Using .map() in order to get the new column result_clean through result_map function

df_copy1['result_clean'] = df_copy1['result'].map(result_map).fillna('OTHER')

# This grouping allows us to visually check if the new categories were correctly classified, which is quickly verified

df_copy1.groupby('result_clean')['result'].value_counts()

result_clean  result                        
APPROVED      APPROVED                          2711
              PARTIAL APPROVE COMPLX STRUCT      974
              APPROVED PARTIAL                   665
              APPRVD PARTIAL/EXTRA INSP REQ      316
              PERM BY AFFDVIT/CORRECT REQ         19
              INSP. & APPROVED BY ENGINEER        16
              INSP. & APPROVED BY ARCHITECT        5
              COMPLETED                            4
              PARTIAL\ APPROVE TRAD PENDING        1
CANCELLED     INSPECTION CANCELLATION            270
              CANCELLATION BY INTERNET           229
              NOT PERFORMED. BAD WEATHER          53
OTHER         NOT INSPECTED                     1103
              NOT READY FOR INSPECTION           413
              CARRY OVER                          96
              INSPECT QTY ASSURANCE PARTIAL       41
              INSPECTION FOR QTY ASSURANCE        40
              FIELD CHECK ONLY                    26
              REFERRED TO REGULATION              10
              UNABLE TO LOCATE/ADD/APT/SPACE       6
              NO PROGRESS                          5
              DEMOLISHED                           4
              VIRTUAL/INSP SEE COMMENT&CALL        4
              INACTIVE                             2
REJECTED      CORRECTIONS REQUIRED               286
              REJECTED NOT ACCESSIBLE            131
              REJECTED NOT TO CODE                92
              REJECTED PLAN & PERMIT MISSING      90
              REJECTED WORK CONCEALED             74
              REJECTED REVISE PLANS REQ           57
              REJECTED , NOTICE COMMENCEMENT      52
              REJECTED APPROVED PLANS REQ.        39
              REJECTED MULTI INSPECTED BLDG        6
              REJECTED FOR POOL BARRIER REQ        3
              REJECTED SUPPLEMENT PERMIT REQ       1
Name: count, dtype: int64

In [252]:
# Finally, with this code we can see that we have converted 35 categories of
# semi-structured free text into 4 well-defined categories, that is,
# we solved the problem of high categorical cardinality

df_copy1['result_clean'].value_counts()

,count
result_clean,
APPROVED,4711
OTHER,1750
REJECTED,831
CANCELLED,552


#### Overcoming the high semantic cardinality of `inspection_type`
The `inspection_type` column suffers from the same problem as `result`, and we will proceed equivalently to reduce the high cardinality of semantic categories in this column.

In [253]:
# Evaluating through the Series .unique() method we see that
# there are 45 different a priori categories in this column

df_copy1['inspection_type'].unique()

array(['ROUGH', 'INSULATION  INSPECTIONS', 'BUCK  AND  FASTENER', 'FINAL',
       'FRAMING/FIRE  STOPPING  WINDOWS', 'TIME  OF  INSTALLATION',
       'BEFORE  ANCHOR  SHEET  IS  COVERED',
       'ROOF  TRUSS/PERM  BRACING/SHEATH', 'ROUGH/SLAB', 'WORK  WITH',
       'POOL  BARRIER', 'POOL  PIPING', 'CAP  SHEET  INSTALLATION',
       'FINAL  ZONING  AND  LANDSCAPE', 'FENCE  FOUNDATION',
       'FINAL  ZONING', 'GROUND  ROUGH', 'TILE  PLACING  (INTERMEDIATE)',
       'TIE  BEAM/REINFORCING', 'SLAB', 'C/O', 'TEMPORARY  C.O.',
       'TEMPORARY  FOR  TESTING', 'FOUNDATION/MONOLITHIC  SLAB',
       'LATHING/FIREWALL/DRYWALL  INSPE', 'PRESSURE  TEST',
       'RECERTIFICATION  SELECTED  QA',
       'SEWER  AND  RELAY/WATER  SERVICE', 'EXPIRED  PERMIT  CHECK',
       'CANCELLATION  REQ/VERIFY  WORK', 'ANNUAL',
       'FOUNDATION  GROUNDING', 'TEMPORARY  FOR  CONSTRUCTION',
       'STEEL  &  MAIN  DRAIN', 'UNDERGROUND',
       'CHECK  SPECIAL  INSPECTOR  LOG', 'SEMI-ANNUAL', 'COMPLAINT',
       

In [254]:
# Standardize the categories in the column to remove double spaces
# at the beginning and end, and convert to uppercase

df_copy1['inspection_type'] = (
    df_copy1['inspection_type']
    .astype(str)
    .str.upper()
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)

  ##### **Standardization criteria documents in the context of inspections**
Below, we'll show the criteria used to group the categories in this column, which might not seem obvious at first glance. It's worth emphasizing that this classification is entirely ad hoc, reducing the initial 45 categories, with a large amount of semi-structured free text, into 15 functional categories:
* **TEMPORARY C.O. → Final**  
   A Temporary Certificate of Occupancy is issued when a building is essentially complete and safe for occupancy, pending only minor items. Even though it is labeled "temporary," its purpose and timing align more closely with a **final** inspection, as it verifies code compliance for habitable conditions.
- **UNDERGROUND → Rough/Structural**  
   Underground inspections review systems such as plumbing, electrical, or foundation components before they are covered. This is part of the **rough/structural** phase, as it involves structural and service elements prior to finishing.
- **TIE BEAM/REINFORCING → Slab/Foundation**  
   Tie beams and reinforcing elements connect and distribute structural loads within the foundation system. They are inspected before or during the slab and foundation pour, making **Slab/Foundation** the correct category.
- **TILE PLACING (INTERMEDIATE)**, **INSULATION INSPECTIONS**, **STEEL & MAIN DRAIN → Framing/Drywall**  
   - *Tile Placing (Intermediate)*: Here it refers to mid-stage installation rather than final finishes.  
   - *Insulation Inspections*: Conducted before closing walls/drywall.  
   - *Steel & Main Drain*: Structural and main drainage inspections before the closure stage.  
   All three occur during the **Framing/Drywall** phase.
- **CAP SHEET INSTALLATION → Roof**  
   The cap sheet is the top layer of a roof's waterproofing system, inspected as part of the **Roof** category.
- **WORK WITH → Temporary/Test**  
   "Work With" inspections are typically coordinated checks alongside other tasks, without closing a permit or construction phase. This makes them more aligned with **Temporary/Test**, as they serve an auxiliary validation purpose.
- **TIME OF INSTALLATION**, **BUCK AND FASTENER → Pre-Construction/Anchors**  
   These inspections check anchoring and fastening components before the main construction proceeds, fitting into **Pre-Construction/Anchors**.
- **TEMPORARY FOR CONSTRUCTION**, **CONSTRUCTION USE ONLY → Temporary/Construction**  
   - **Temporary/Construction**: Temporary approvals for specific uses during the construction process (e.g., enabling a construction area for worker use, temporary utilities).  
   - **Temporary/Test**: Reserved for technical validation or performance testing.  
   These two cases clearly belong in **Temporary/Construction** rather than Temporary/Test.
    
- The decision to consolidate `inspection_description` into **15 functional categories** was made to balance interpretability and analytical value:

 1. **Optimal granularity**: Too few categories would oversimplify the data and obscure important patterns, while too many would fragment the analysis and complicate visualizations.
 2. **Industry practice**: Grouping into 10–20 categories is common in professional reporting to ensure categories remain understandable while preserving specificity.
 3. **Future analysis compatibility**: The categories are sufficiently distinct to capture different approval/rejection patterns in the upcoming EDA and predictive modeling stages.
 4. **Avoiding overcompression**: Categories like "Roof" and "Slab/Foundation" were intentionally kept separate due to different technical contexts and performance outcomes.
 5. **Auditability**: With 15 categories, manual review remains manageable, ensuring classification errors can be detected and corrected efficiently.


In [255]:
# Built this sort function to group categories in this column

def classifying_inspections(S):

    if S in ['FINAL', 'FINAL ZONING AND LANDSCAPE', 'FINAL ZONING', 'TEMPORARY C.O.']:
        return 'Final'

    elif S in ['ROUGH', 'ROUGH/SLAB', 'GROUND ROUGH', 'UNDERGROUND', 'ROUGH/PRESSUR TEST']:
        return 'Rough/Structural'

    elif S in ['SLAB', 'FOUNDATION/MONOLITHIC SLAB', 'TIE BEAM/REINFORCING',
                  'FENCE FOUNDATION', 'FOUNDATION GROUNDING']:
        return 'Slab/Foundation'

    elif S in ['FRAMING/FIRE STOPPING WINDOWS', 'LATHING/FIREWALL/DRYWALL INSPE',
                  'TILE PLACING (INTERMEDIATE)', 'INSULATION INSPECTIONS',
                  'STEEL & MAIN DRAIN']:
        return 'Framing/Drywall'

    elif S in ['ROOF TRUSS/PERM BRACING/SHEATH', 'CAP SHEET INSTALLATION']:
        return 'Roof'

    elif S in ['SEWER AND RELAY/WATER SERVICE', 'POOL PIPING', 'POOL BARRIER',
                  'PRESSURE TEST']:
        return 'Plumbing/Pool/Service'

    elif S == 'C/O':
        return 'Occupancy Cert.'

    elif S in ['TEMPORARY FOR TESTING', 'TEMPORARY FOR CONSTRUCTION', 'WORK WITH']:
        return 'Temporary/Test'

    elif S in ['BEFORE ANCHOR SHEET IS COVERED', 'TIME OF INSTALLATION',
                  'BUCK AND FASTENER']:
        return 'Pre-Construction/Anchors'

    elif S in ['ANNUAL', 'SEMI-ANNUAL', 'CHECK SPECIAL INSPECTOR LOG',
                  'RECERTIFICATION SELECTED QA', 'CANCELLATION REQ/VERIFY WORK',
                  'FIELD CHECK', 'EXPIRED PERMIT CHECK']:
        return 'Administrative/Other'

    elif S in ['COMPLAINT', 'FIRE DAMAGE']:
        return 'Complaint/Incident-Based'

    elif S in ['TEMPORARY FOR CONSTRUCTION', 'CONSTRUCTION USE ONLY']:
        return 'Temporary/Construction Use'

    elif S in ['VERIFY COMPLETION OF EVENT', 'VERIFICATION OF EXISTING UNIT']:
        return 'Verification/Closeout'

    elif S in ['PARTIAL/IN PROGRESS SOIL IMPR']:
        return 'Soil/Foundation Work'

    else:
        return 'Other'


In [256]:
# Create a new column to save the inspection types already classified into 15 functional categories

df_copy1['inspection_clean'] = df_copy1['inspection_type'].apply(classifying_inspections)

# With this .groupby() we can visually check that the groupings were done correctly

df_copy1.groupby('inspection_clean')['inspection_type'].value_counts()

inspection_clean            inspection_type               
Administrative/Other        ANNUAL                              72
                            CHECK SPECIAL INSPECTOR LOG         68
                            RECERTIFICATION SELECTED QA         64
                            CANCELLATION REQ/VERIFY WORK        21
                            EXPIRED PERMIT CHECK                 9
                            SEMI-ANNUAL                          2
                            FIELD CHECK                          1
Complaint/Incident-Based    FIRE DAMAGE                          2
                            COMPLAINT                            1
Final                       FINAL                             2572
                            TEMPORARY C.O.                     106
                            FINAL ZONING AND LANDSCAPE          42
                            FINAL ZONING                        20
Framing/Drywall             FRAMING/FIRE STOPPING WINDOWS      422
                            LATHING/FIREWALL/DRYWALL INSPE     186
                            INSULATION INSPECTIONS             131
                            TILE PLACING (INTERMEDIATE)         98
                            STEEL & MAIN DRAIN                  32
Occupancy Cert.             C/O                                127
Plumbing/Pool/Service       SEWER AND RELAY/WATER SERVICE       95
                            PRESSURE TEST                       58
                            POOL BARRIER                        57
                            POOL PIPING                         40
Pre-Construction/Anchors    BEFORE ANCHOR SHEET IS COVERED     386
                            TIME OF INSTALLATION               260
                            BUCK AND FASTENER                  247
Roof                        ROOF TRUSS/PERM BRACING/SHEATH      98
                            CAP SHEET INSTALLATION              90
Rough/Structural            ROUGH                             1514
                            GROUND ROUGH                       154
                            ROUGH/SLAB                          73
                            UNDERGROUND                         35
                            ROUGH/PRESSUR TEST                   1
Slab/Foundation             SLAB                               222
                            TIE BEAM/REINFORCING               165
                            FOUNDATION/MONOLITHIC SLAB         152
                            FENCE FOUNDATION                    87
                            FOUNDATION GROUNDING                42
Soil/Foundation Work        PARTIAL/IN PROGRESS SOIL IMPR        1
Temporary/Construction Use  CONSTRUCTION USE ONLY                1
Temporary/Test              WORK WITH                           43
                            TEMPORARY FOR TESTING               39
                            TEMPORARY FOR CONSTRUCTION           5
Verification/Closeout       VERIFICATION OF EXISTING UNIT        2
                            VERIFY COMPLETION OF EVENT           1
Name: count, dtype: int64

##### **Consolidation of Rare Categories**
When analyzing this result of `.groupby()` we realize that four functional categories `Verification/Closeout`, `Temporary/Construction Use`, `Soil/Foundation Work`, `Complaint/Incident-Based`, together accounted for only 8 inspections (`0.1%` of total). They were merged into the `Other` category to:
- Reduce categorical sparsity
- Simplify visualizations and reporting
- Avoid overemphasizing rare, non-critical cases

Finally, we have 11 functional categories within the categorical column `inspection_clean`

In [257]:
# Merge rare categories into the 'Other' category using the .apply() method

rare_categories = ['Verification/Closeout', 'Temporary/Construction Use', 'Soil/Foundation Work', 'Complaint/Incident-Based']
df_copy1['inspection_clean'] = df_copy1['inspection_clean'].apply(lambda x: 'Other' if x in rare_categories else x)

# With this .loc we can visually check that the merge was done correctly

df_copy1.loc[ df_copy1['inspection_clean'] == 'Other', 'inspection_type' ].value_counts()


,count
inspection_type,
FIRE DAMAGE,2
VERIFICATION OF EXISTING UNIT,2
VERIFY COMPLETION OF EVENT,1
COMPLAINT,1
PARTIAL/IN PROGRESS SOIL IMPR,1
CONSTRUCTION USE ONLY,1


In [258]:
# Observe how the data frame is looking so far!

df_copy1.head(8)

,date,permit_number,sequence_number,input_order,inspection_type,address,permit,result,inspector,result_clean,inspection_clean
0,2025/06/05 04:00:00+00,2025006752,1,30,ROUGH,1611 NW 12 AVE,MECH,NOT INSPECTED,Eusebio Aguilar-Godoy,OTHER,Rough/Structural
1,2025/06/05 04:00:00+00,2023056382,27,194,INSULATION INSPECTIONS,1700 NE 145 ST,BLDG,NOT INSPECTED,Alexander Bedoya,OTHER,Framing/Drywall
2,2025/06/05 04:00:00+00,2024058290,3,27,ROUGH,3100 SW 62 AVE,MECH,NOT INSPECTED,Luigi Chinea,OTHER,Rough/Structural
3,2025/06/05 04:00:00+00,2024056228,4,99,ROUGH,1611 NW 12 AVE,ELEC,NOT INSPECTED,Lawrence Molina,OTHER,Rough/Structural
4,2025/06/05 04:00:00+00,2025028446,7,89,BUCK AND FASTENER,9907 SW 164 PL,BLDG,NOT INSPECTED,Angel Castro,OTHER,Pre-Construction/Anchors
5,2025/06/05 04:00:00+00,2024073267,2,4,ROUGH,7389 SW 45 ST,PLUM,NOT INSPECTED,Barnette K Gunn,OTHER,Rough/Structural
6,2025/06/05 04:00:00+00,2024022578,1,46,ROUGH,1611 NW 12 AVE,PLUM,NOT INSPECTED,Kevin OConnor,OTHER,Rough/Structural
7,2025/06/05 04:00:00+00,2025046727,8,22,FINAL,3400 SW 100 AVE,MECH,NOT INSPECTED,Luigi Chinea,OTHER,Final


### **Handling the column `date`**
We already mentioned that the `date` column most likely contains the time as its default value. We'll convert the data in this column to Pandas' `datetime` format and remove the time.

In [259]:
# We convert the format of the `date` column to `datetime`,
# and we are left with only the date, that is, year, month and day of the month

df_copy1['date'] = pd.to_datetime(df_copy1['date']).dt.date

# With the Series .value_counts() method we will check the distribution
# of the dates, and if there is any non-standard value at first glance

df_copy1['date'].value_counts()

,count
date,
2025-05-29,1480
2025-05-30,1472
2025-06-03,1378
2025-06-02,1321
2025-06-05,1106
2025-06-04,1083
2025-05-31,4


### **Discovering the ZIP code for addresses through a Google geocoding API**
The `zip` column has been integrated into the dataframe after a geocoding process based on the `address` column using the Google Maps API. The code that performed this enrichment is documented in the `geocoding-public.ipynb` technical notebook. This step has now been completed, so the `zip` column is ready to be used directly.

* To work with the address section of our `df_copy1` dataframe, we apply the `normalizing` function to this column to remove double spaces and convert everything to uppercase.
* We then save the column as a `DataFrame` object, which we export as a `.csv` file for use in the other geocoding notebook.
* The final dataframe obtained in the other notebook, containing each address (in the same order as `df_copy1`) with its zip code found, was then imported into our environment using the `url` generated in the GitHub repository, and the new `zip` column was then included in our dataframe.

In [260]:
# Applying the 'normalizing' function to the column `address`

df_copy1['address'] = df_copy1['address'].apply(normalizing)

# Exporting the column `address` as .csv file in order to use it in geocoding process

df_address = pd.DataFrame(df_copy1.address)
df_address.to_csv('solo_address.csv', index=False)

In [261]:
# Loading the addresses enriched directly from the raw github url

url = 'https://raw.githubusercontent.com/frankrgamboa/miami-inspections-cleaning/refs/heads/main/address_ZIP.csv'

# Reading the .csv file, making sure the `zip` column contains str instead of floats or ints

address_zip = pd.read_csv(url, dtype = ({'zip' : str}))

# Joining the new data obtained in the new column `zip`

df_copy1['zip'] = address_zip['zip']

# Preview the modified dataframe

df_copy1.head(8)

,date,permit_number,sequence_number,input_order,inspection_type,address,permit,result,inspector,result_clean,inspection_clean,zip
0,2025-06-05,2025006752,1,30,ROUGH,1611 NW 12 AVE,MECH,NOT INSPECTED,Eusebio Aguilar-Godoy,OTHER,Rough/Structural,33136
1,2025-06-05,2023056382,27,194,INSULATION INSPECTIONS,1700 NE 145 ST,BLDG,NOT INSPECTED,Alexander Bedoya,OTHER,Framing/Drywall,33181
2,2025-06-05,2024058290,3,27,ROUGH,3100 SW 62 AVE,MECH,NOT INSPECTED,Luigi Chinea,OTHER,Rough/Structural,33155
3,2025-06-05,2024056228,4,99,ROUGH,1611 NW 12 AVE,ELEC,NOT INSPECTED,Lawrence Molina,OTHER,Rough/Structural,33136
4,2025-06-05,2025028446,7,89,BUCK AND FASTENER,9907 SW 164 PL,BLDG,NOT INSPECTED,Angel Castro,OTHER,Pre-Construction/Anchors,33196
5,2025-06-05,2024073267,2,4,ROUGH,7389 SW 45 ST,PLUM,NOT INSPECTED,Barnette K Gunn,OTHER,Rough/Structural,33314
6,2025-06-05,2024022578,1,46,ROUGH,1611 NW 12 AVE,PLUM,NOT INSPECTED,Kevin OConnor,OTHER,Rough/Structural,33136
7,2025-06-05,2025046727,8,22,FINAL,3400 SW 100 AVE,MECH,NOT INSPECTED,Luigi Chinea,OTHER,Final,34476


### **Final comments**
The dataset is clean and ready for further analysis. The following comments are valid:
- The dataframe currently has 12 columns, down from the initial 20. Of these 12, two have not been eliminated at this stage of the analysis, but their information has been categorized into two newly created columns, `result_clean` and `inspection_clean`. Therefore, only 10 of these 12 columns are functional.
- The `permit_number`, `sequence_number`, and `input_order` columns remain, but should be analyzed in later stages to understand their structure.
 - `sequence_number` appears to store steps of the inspection process encoded in numbers.
 * The `input_order` column may contain some sort of priority order based on when the inspection request was made.
 *The `permit_number` values associated with each inspection most likely store information about the permit type and inspection classification. It may be highly correlated with at least `permit` and `inspection_clean`; it is recommended to study it before including it in explanatory or predictive analyses to avoid bias or overfitting.
-The `date` column only holds records for a full calendar week; it may therefore be of little use.
-The `address` and `zip` columns store information about the location of the inspected sites.
-The `result_clean`, `inspection_clean`, and `permit` columns have relatively few categories, the first two being the result of the classification process we performed to address high semantic cardinality. This means they are well-suited to future exploratory analysis, logistic regression, and visualizations.

In [264]:
# We save the cleaned data frame in a new variable and export it to our environment

df_copy1.to_csv('inspections-cleaned.csv', index = False)